In [1]:
#import all library
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

### Data Set

1. Housing Resales Price from 2012 to 2021

In [2]:
#Read CSV file
housing_train = pd.read_csv('../Data/train.csv')
housing_test = pd.read_csv('../Data/test.csv')
singapore_regions = pd.read_excel('../Data/Singapore_Regions.xlsx')

C:\Users\Jia Hui\AppData\Local\Temp\ipykernel_8572\1389411070.py:2: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  housing_train = pd.read_csv('../Data/train.csv')
C:\Users\Jia Hui\AppData\Local\Temp\ipykernel_8572\1389411070.py:3: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  housing_test = pd.read_csv('../Data/test.csv')


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
housing_train['train'] = 1
housing_test['train'] = 0

housing = pd.concat([housing_train,housing_test])

In [ ]:
#Display top 5 rows
housing.head()

In [ ]:
#Display top 5 rows
singapore_regions.head()

In [ ]:
housing.shape

In [ ]:
#Check for null values
print(housing[housing['train']==1].isnull().sum())

In [ ]:
missing_value_percent = (829/150634) * 100 
missing_value_percent

Dropping the missing values as it is less than 1%

### Data Cleaning

In [ ]:
# Change all the categorical boolean  to 1 and 0 
boolean_cols = ['commercial','market_hawker','multistorey_carpark','precinct_pavilion']
for col in boolean_cols:
    housing[col] =  housing[col].map({'Y':1,'N':0})

#Change Na values in Mall_Within_* and hawker_Within_* column to 0 
cols = ['Mall_Within', 'Hawker_Within']
distance_list = ['_500m', '_1km', '_2km']
new_cols = []

# Generate new column names and fill NaN values with zeros
for col in cols:
    for distance in distance_list:
        new_col = col + distance
        new_cols.append(new_col)

for col in new_cols:
    housing[col] = housing[col].fillna(0)

    
#Drop Na Valus for train = 1 only (Train Dataset)
housing['indicator_to_drop'] = np.where((housing['train'] == 1) & (housing['Mall_Nearest_Distance'].isna()), 1, 0)
housing = housing[housing['indicator_to_drop'] !=1 ]


# Include Regions ans Mature indicator from singapore_regions df to housing df by left join
housing = housing.merge(singapore_regions, how = 'left', left_on = 'planning_area' , right_on = 'area')

# Drop Unnecessary columns
# Drop residential because all the column is Y
# block, street_name, address, postal code can be replaced by lat, long, and town
# lease_commence_date and year_completed can be replaced by hbd_age
# lower, upper, mid can be replaced by storey_range and mid_storey
# Drop floor_area_sqm as it is a convertion unit from floor_area_sqft

housing.drop(columns = ['residential','area','town','block','street_name','lease_commence_date','lower','upper','mid','address','postal','year_completed','floor_area_sqm'],inplace = True)


# Change all column to lower case
housing.columns = housing.columns.str.lower()

# Create one more column to indicate top 10 primary school (1,0)
top_10_primary_school = ['Nanyang Primary School','Catholic High School','Tao Nan School','Nan Hua Primary School','St. Hilda’s Primary School','Henry Park Primary School','Anglo-Chinese School Primary','Raffles Girls’ Primary School','Pei Hwa Presbyterian Primary School','CHIJ St. Nicholas Girls’ School']
housing['top_10_pri_sch'] = np.where(housing['pri_sch_name'].isin(top_10_primary_school) , 1, 0)

# Create one more column to rank sec_sch_name based on cutoff_point
housing['cutoff_point_rank'] = housing['cutoff_point'].rank(ascending=False, method='dense')
housing.drop(columns = ['cutoff_point'],inplace = True)


In [ ]:
housing[housing['train']==0].head()

In [ ]:
#Check for null values
print(housing[housing['train']==1].isnull().sum())

In [ ]:
#Train on non na value 
#However, fill na value for Mall_Nearest_Distance when train = 0 (Test Dataset)

# Create an instance of KNNImputer
imputer = KNNImputer(n_neighbors=5)


X = housing[['mall_nearest_distance']]

# Fit the imputer to your data and transform it
X_imputed = imputer.fit_transform(X)

# Replace the original 'mall_nearest_distance' column in the DataFrame with the imputed values
housing['mall_nearest_distance'] = X_imputed

In [ ]:
#Check for null values
print(housing[housing['train']==0].isnull().sum())

In [ ]:
housing[housing['train']==0].head()

In [ ]:
#Save to csv file
housing[housing['train']==1].to_csv('../data/cleaned_train_data.csv', index=False)

In [ ]:
# Drop lat and long related columns for regression models
cols = housing.columns
subtext = ['latitude','longitude']

cols_to_drop = [col for col in cols if any(sub in col for sub in subtext)]
housing_2 = housing.drop(columns = cols_to_drop , inplace = False)

In [ ]:
#Save to csv file
housing_2.to_csv('../data/cleaned_combined_data_wo_lat_long.csv', index=False)